In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (accuracy_score, precision_score,
                             recall_score, f1_score, classification_report, 
                             ConfusionMatrixDisplay)

from sklearn import set_config
set_config(transform_output="pandas")

from category_encoders import HashingEncoder

import sqlite3

from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import make_pipeline

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, ConfusionMatrixDisplay

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import roc_auc_score, RocCurveDisplay, PrecisionRecallDisplay

import joblib



In [ ]:
# # First 	1	Enable GPU Runtime:
# 	•	Go to "Runtime" -> "Change runtime type".
# 	•	Select "GPU" as the hardware accelerator.
# 	•	Click "Save".

In [ ]:
# # Install a specific CUDA version
# !apt-get --purge remove cuda nvidia* libnvidia-*
# !dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
# !apt-get remove cuda-*
# !apt autoremove
# !apt-get update

# # Install CUDA 11.x
# !wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
# !mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
# !wget https://developer.download.nvidia.com/compute/cuda/11.8.0/local_installers/cuda-repo-ubuntu1804-11-8-local_11.8.0-520.61.05-1_amd64.deb
# !dpkg -i cuda-repo-ubuntu1804-11-8-local_11.8.0-520.61.05-1_amd64.deb
# !cp /var/cuda-repo-ubuntu1804-11-8-local/cuda-*-*.pub /tmp/cuda-*-*.pub
# !apt-key add /tmp/cuda-*-*.pub
# !apt-get update
# !apt-get -y install cuda

In [ ]:
# !pip install cudf-cu11 --extra-index-url=https://pypi.nvidia.com

In [ ]:
# import cudf  # this should work without any errors

In [ ]:
# %load_ext cudf.pandas
# # Google Colab: Add `%load_ext cudf.pandas` before importing pandas to speed up operations using GPU (musr first choose runtime that is a GPU)

In [ ]:
# saving preprocessor for future use
to_save = {"preprocessor" : preprocessor,
           "porch_func" : porch_func}
filename = "preprocessor.joblib"

joblib.dump(to_save, filename)

In [ ]:
# # Clone the entire repo.
# !git clone -l -s git://github.com/jakevdp/PythonDataScienceHandbook.git cloned-repo
# %cd cloned-repo
# !ls

In [ ]:
# # Fetch a single <1MB file using the raw GitHub URL.
# !curl --remote-name \
#      -H 'Accept: application/vnd.github.v3.raw' \
#      --location https://api.github.com/repos/jakevdp/PythonDataScienceHandbook/contents/notebooks/data/california_cities.csv

In [ ]:
# Run only once!
import os
print(os.getcwd())
os.chdir('../..')
print(os.getcwd())

In [ ]:
local_prepend = "../.."
input_path = "/kaggle/input/home-credit-default-risk/"
# kaggle_notebook_input_path = "/kaggle/input/home-credit-default-risk/"

# con = sqlite3.connect(local_prepend + input_path + "home-credit-default-risk.sqlite") # connect to database
# pd.read_csv(local_prepend + input_path + "application_train.csv").to_sql("application_train", con, if_exists='append', index=False, index_label="SK_ID_CURR")

# application_train = pd.read_csv(local_prepend + input_path + "application_train.csv")

# bureau_balance = pd.read_csv(local_prepend + input_path + "bureau_balance.csv")
# bureau = pd.read_csv(local_prepend + input_path + "bureau.csv")
# credit_card_balance = pd.read_csv(local_prepend + input_path + "credit_card_balance.csv")
# installments_payments = pd.read_csv(local_prepend + input_path + "installments_payments.csv")
# POS_CASH_balance = pd.read_csv(local_prepend + input_path + "POS_CASH_balance.csv")
# previous_application = pd.read_csv(local_prepend + input_path + "previous_application.csv")

# Description of columns in provided datasets
HomeCredit_columns_description = pd.read_csv(local_prepend + input_path + "HomeCredit_columns_description.csv", encoding = "latin")

# # Test Data for later use
# application_test = pd.read_csv(local_prepend + input_path + "application_test.csv")

In [ ]:
HomeCredit_columns_description

In [ ]:
HomeCredit_columns_description[HomeCredit_columns_description["Table"] == "application_{train|test}.csv"][["Row", "Description"]]

In [ ]:
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_colwidth", None)

In [ ]:
HomeCredit_columns_description[~HomeCredit_columns_description["Special"].isna()]

In [ ]:

# dfs_to_merge = [bureau, previous_application, POS_CASH_balance, credit_card_balance]

# for df in dfs_to_merge:
#     application_train = pd.merge(application_train, df, how="inner", on="SK_ID_CURR")

# application_train.shape

In [ ]:
application_train = pd.merge(application_train, bureau, how="left", on="SK_ID_CURR")
application_train = application_train.drop_duplicates(subset=["SK_ID_CURR"])
application_train = pd.merge(application_train, previous_application, how="left", on="SK_ID_CURR")
application_train = application_train.drop_duplicates(subset=["SK_ID_CURR"])
application_train = pd.merge(application_train, POS_CASH_balance, how="left", on="SK_ID_CURR")
application_train = application_train.drop_duplicates(subset=["SK_ID_CURR"])
application_train = pd.merge(application_train, credit_card_balance, how="left", on="SK_ID_CURR")
application_train = application_train.drop_duplicates(subset=["SK_ID_CURR"])

In [ ]:
application_train.select_dtypes(include='object')

In [ ]:
object_cols = application_train.select_dtypes(include='object').columns.to_list()

for col in object_cols:
    print(application_train[col].value_counts())

In [ ]:
df_num = application_train.select_dtypes(include='number')
df_num

In [ ]:
# df_num = df_num.dropna()
# df_num

In [ ]:
df_num = df_num.drop_duplicates(subset=["SK_ID_CURR"])
df_num

In [ ]:
# Generate a mask for the upper triangle
mask = np.zeros_like(df_num.corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(df_num.corr().where(lambda x: abs(x)>0.5), mask=mask, square=True)

In [ ]:
# split data
X = df_num.drop(columns=["TARGET", "SK_ID_CURR"])
y = df_num["TARGET"].copy()

In [ ]:
# Training split:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

num_cols = X.columns.to_list()

# Create Transformers
zero_imputer = SimpleImputer(strategy="constant", fill_value=0)
# scaler = StandardScaler()

# Create tuples
impute_tuple = ("imputation", zero_imputer, num_cols)
# scaler_tuple = ("scaling", scaler, num_cols)


# Create ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[impute_tuple], remainder="passthrough",
    verbose_feature_names_out=True)

# Test preprocessor
preprocessor.fit(X_train)
X_train_proc = preprocessor.transform(X_train)
X_test_proc = preprocessor.transform(X_test)
X_val_proc = preprocessor.transform(X_val)

In [ ]:
logreg_balanced = LogisticRegression(max_iter=1000, class_weight="balanced")

logreg_balanced_pipe = make_pipeline(StandardScaler(), logreg_balanced)

logreg_balanced_pipe.fit(X_train_proc, y_train)

# balanced_results = eval_classification(logreg_balanced_pipe, X_train_proc, y_train, X_test_proc, y_test,
#                               model_name="Logistic Regression Balanced Class Weights")

# balanced_results

print("Train")
print(classification_report(y_train, train_pred))
print("Test")
print(classification_report(y_test, test_pred))

In [ ]:
# from sklearn.model_selection import GridSearchCV

# knn = KNeighborsClassifier()
# param_space = {'n_neighbors': range(3,8)}
# knn_gs = GridSearchCV(knn, param_space)
# knn_gs.fit(X_train_proc, y_train)
# best_knn = knn_gs.best_estimator_
# best_knn

In [ ]:
knn_pipe = Pipeline([("scaler", StandardScaler()), ("knn", best_knn)])

knn_pipe.fit(X_train_proc, y_train)

In [ ]:
zero_imputer.fit(X)

zero_X = zero_imputer.transform(X)
knn_pipe2 = Pipeline([("scaler", StandardScaler()), 
                      ("knn", KNeighborsClassifier(n_neighbors=6))])

knn_pipe2.fit(zero_X, y) 

zero_X_cols = zero_X.columns.to_list()


In [ ]:
test_data_file = "application_test.csv"
test_data = pd.read_csv(local_prepend + input_path + test_data_file)

test_data = pd.merge(test_data, bureau, how="left", on="SK_ID_CURR")
test_data = test_data.drop_duplicates(subset=["SK_ID_CURR"])
test_data = pd.merge(test_data, previous_application, how="left", on="SK_ID_CURR")
test_data = test_data.drop_duplicates(subset=["SK_ID_CURR"])
test_data = pd.merge(test_data, POS_CASH_balance, how="left", on="SK_ID_CURR")
test_data = test_data.drop_duplicates(subset=["SK_ID_CURR"])
test_data = pd.merge(test_data, credit_card_balance, how="left", on="SK_ID_CURR")
test_data = test_data.drop_duplicates(subset=["SK_ID_CURR"])

ids = test_data.pop("SK_ID_CURR")

test_data = test_data[zero_X_cols]

zero_test = zero_imputer.transform(test_data)

preds = knn_pipe2.predict_proba(zero_test)[:,1]

output = pd.DataFrame({"SK_ID_CURR": ids,
                       "TARGET": preds})

In [ ]:
output

In [ ]:
sample_submission_file = "sample_submission.csv"
sample_submission_df = pd.read_csv(local_prepend + input_path + sample_submission_file)
sample_submission_df["TARGET"] = preds
sample_submission_df.to_csv("00f_more_merges.csv", index=False)
sample_submission_df.head()

In [ ]:
# # create a logistic regression
# log_reg = LogisticRegression(max_iter=100000, random_state=42)
# log_reg.fit(X_train_proc, y_train)
# log_reg.predict_proba(X_test_proc)[:,0]

In [ ]:
eval_classification(best_knn, X_train_proc, y_train, X_test_proc, y_test, model_name="best knn", results_frame=None,
                        pos_label=1, average="binary")

In [ ]:
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import make_pipeline

In [ ]:
y_train.value_counts()